In [1]:
import import_ipynb
import matplotlib.pyplot as plt
from FULL_DATA import final_df
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords

importing Jupyter notebook from FULL_DATA.ipynb
importing Jupyter notebook from Create_Labels.ipynb
(6383, 4)
4402
(4402, 7)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Making Labels
final_df['SENTIMENT'] = [0 if (x > 1 and x<3.4999) else 1 if (x > 3.5 and x<4.49999) else -1  for x in final_df['ratings']]
print(final_df['SENTIMENT'].value_counts())
print(final_df.shape)
final_df.head()

 1    2367
 0    1144
-1     866
Name: SENTIMENT, dtype: int64
(4377, 8)


,TRANSCRIPTS,TICKER,TRANSCRIPT_DATE,RATINGS_DATE,ratings,DATE_DIFF,KEY,SENTIMENT
14602,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-07-31,2018-09-28,4.714,59 days,A_2018-09-28,-1
14599,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-11-19,2018-12-31,4.571,42 days,A_2018-12-31,-1
14592,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-01-31,2019-03-29,5.000,57 days,A_2019-03-29,-1
14589,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-05-14,2019-06-28,4.867,45 days,A_2019-06-28,-1
14586,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-08-14,2019-09-30,4.286,47 days,A_2019-09-30,1


In [3]:
final_df['SENTIMENT'].value_counts()

 1    2367
 0    1144
-1     866
Name: SENTIMENT, dtype: int64

In [4]:
# iterate through each sentence in the file 
data = [] 
for i in final_df['TRANSCRIPTS']: 
    temp = [] 
    # tokenize the sentence into words 
#     print(i)
    for j in word_tokenize(i): 
        if j in temp:
            pass
        elif j in stopwords.words('english'):
            pass
        else:
            temp.append(j.lower()) 
    data.append(temp)         
#data

In [5]:
tf=TfidfVectorizer(lowercase=True,max_df = .9,min_df=.1,ngram_range = (1,1))
text_tf= tf.fit_transform(final_df['TRANSCRIPTS'])
tfidf = dict(zip(tf.get_feature_names(), tf.idf_))
# tfidf

In [6]:
#Word2Vec
max_len = 500
from gensim.models import Word2Vec
word2vec = Word2Vec(data, min_count=2,size = max_len, window = 5)
vocabulary = word2vec.wv.vocab

In [7]:
avg_list = []   
import numpy as np
for i in final_df['TRANSCRIPTS']: 
    vec = np.zeros(max_len).reshape((1, max_len))
    count = 0
#     print("iiiiiiiiiiiiiiiiiiiiii",i)
    for j in word_tokenize(i):
#         print(j)
        try:
            vec += word2vec[j].reshape((1, max_len)) * tfidf[j]
            count += 1.
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    avg_list.append(vec[-1])

C:\Users\Chad\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [9]:
import pandas as pd
X_train, X_test, y_train, y_test = train_test_split(
     pd.DataFrame(avg_list), final_df['SENTIMENT'], test_size=0.15, random_state=1)

In [10]:
from keras.utils import to_categorical
num_classes = 3

y_train_adjusted = to_categorical(np.array(y_train), num_classes = num_classes)
y_test_adjusted = to_categorical(np.array(y_test), num_classes = num_classes)

Using TensorFlow backend.


In [11]:
#Simple Feed-Forward
import tensorflow 
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD

epochs = 1500
#lr =.1
num_classes = 3
model = Sequential()
model.add(Dense(100, activation='sigmoid', input_dim=max_len))
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation = 'softmax'))

#sgd = SGD(lr=0.05, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
model.fit(np.array(X_train), y_train_adjusted, epochs=epochs, batch_size=32, verbose=0,validation_data=(np.array(X_test), y_test_adjusted), shuffle=False)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
from sklearn import metrics

In [13]:
score = model.evaluate(X_test, y_test_adjusted, batch_size=32, verbose=2)

y_prob = model.predict(X_test)
predicted = np.argmax(y_prob, axis = 1)

sklearn_y_test = np.argmax(y_test_adjusted, axis = 1)

print("Making Sure Accuracy is the same:",metrics.accuracy_score(sklearn_y_test, predicted))
print("Feed-Foward Precision:",metrics.precision_score(sklearn_y_test, predicted, average = 'weighted'))
print("Feed-Foward Recall:",metrics.recall_score(sklearn_y_test, predicted, average = 'weighted'))
print("Feed-Foward F1:",metrics.f1_score(sklearn_y_test, predicted, average = 'weighted'))

 - 0s - loss: 3.3570 - acc: 0.6986
Making Sure Accuracy is the same: 0.6986301369863014
Feed-Foward Precision: 0.6933117021780927
Feed-Foward Recall: 0.6986301369863014
Feed-Foward F1: 0.695207910276481


In [14]:
#For Report - Baseline
'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final_df['TRANSCRIPTS'], final_df['SENTIMENT'], test_size=0.15, random_state=1)

print(y_train)
print()
'''
                                                            
predicted_baseline = np.full(y_test.size, 1)

print("Baseline Accuracy",metrics.accuracy_score(y_test, predicted_baseline))
print("Baseline Precision:",metrics.precision_score(y_test, predicted_baseline, average = 'weighted'))
print("Baseline Recall:",metrics.recall_score(y_test, predicted_baseline, average = 'weighted'))
print("Baseline F1:",metrics.f1_score(y_test, predicted_baseline, average = 'weighted'))

Baseline Accuracy 0.5601217656012176
Baseline Precision: 0.31373639230022543
Baseline Recall: 0.5601217656012176
Baseline F1: 0.40219475071463046


C:\Users\Chad\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
#Printing Confusion Matrix - For Report
print("Confusion Matrix DL:")
print(metrics.multilabel_confusion_matrix(sklearn_y_test, predicted))

Confusion Matrix DL:
[[[436  59]
  [ 56 106]]

 [[199  90]
  [ 75 293]]

 [[481  49]
  [ 67  60]]]
